In [7]:
"""
    Analyze the nifti datasets for MONAI parameter adjustments
"""
import os
import nibabel as nib
import numpy as np
from glob import glob

def analyze_nii_dataset(volume_paths):
    # analyze voxel size, shape, and intensity based on input data
    voxel_sizes = []
    shapes = []
    for path in volume_paths:
        img = nib.load(path)
        data = img.get_fdata()
        voxel_sizes.append(img.header.get_zooms())
        shapes.append(data.shape)

    mean_spacing = np.mean(voxel_sizes, axis=0)
    mean_shape = np.mean(shapes, axis=0)

    pixdim = tuple(round(s, 2) for s in mean_spacing)
    a_min, a_max = -200, 250

    # estimate spatial_size and batch_size based on GPU memory
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        mem_free = max([gpu.memoryFree for gpu in gpus])  # in MB
    except Exception:
        mem_free = 0  # CPU

    # configure based on memory (very rough estimates)
    if mem_free >= 20000:
        spatial_size = [256, 256, 256]
        batch_size = 2
    elif mem_free >= 10000:
        spatial_size = [192, 192, 128]
        batch_size = 1
    elif mem_free >= 4000:
        spatial_size = [128, 128, 64]
        batch_size = 1
    else:
        spatial_size = [96, 96, 64]
        batch_size = 1  # fallback for low-mem or CPU

    return {
        'pixdim': pixdim,
        'a_min': a_min,
        'a_max': a_max,
        'spatial_size': spatial_size,
        'batch_size': batch_size,
        'mem_free_MB': mem_free
    }

volume_paths = glob("/kaggle/input/**/volume-*.nii", recursive=True)
params = auto_configure_parameters(volume_paths)

# print("analyze_nii_dataset:")
# for k, v in params.items():
#     print(f"{k}: {v}")

Auto-configured parameters:
pixdim: (0.79, 0.79, 1.51)
a_min: -200
a_max: 250
spatial_size: [96, 96, 64]
batch_size: 1
mem_free_MB: 0


In [ ]:
import os
from glob import glob

"""
    Get data inputs, assumes CT volumes and segmentation masks have corresponding names and indices.
    :param str in_dir: file path of data.
"""
def prepare(in_dir="/kaggle/input"):

    volume_dict = {}
    segmentation_dict = {}
    
    # find all .nii files under in_dir
    nii_files = glob(os.path.join(in_dir, "**", "*.nii"), recursive=True)
    
    for filepath in nii_files:
        filename = os.path.basename(filepath)
        if filename.startswith("volume-"):
            idx = int(filename.split("-")[1].split(".")[0])
            volume_dict[idx] = filepath
        elif filename.startswith("segmentation-"):
            idx = int(filename.split("-")[1].split(".")[0])
            segmentation_dict[idx] = filepath

    # match volume and segmentation by idx
    matched_keys = sorted(set(volume_dict.keys()) & set(segmentation_dict.keys()))
    all_files = [{"vol": volume_dict[k], "seg": segmentation_dict[k]} for k in matched_keys]

    # split into 80 train and 20 test
    split_idx = int(0.8 * len(all_files))
    train_files = all_files[:split_idx]
    test_files = all_files[split_idx:]

    return train_files, test_files

In [ ]:
"""
Written by Haifaa with sources:
https://github.com/Project-MONAI/tutorials
"""

import re
from glob import glob
from monai.transforms import (
    Compose,
    EnsureChannelFirstD,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

"""
    Use MONAI transforms to prepares data for segmentation.
    Voxel: 3D grid representation of data.
    
    :param tuple pixdim: standard voxel spacing (in millimeters) for resampling the images in the x, y, and z dimensions.
    :param int a_min: intensity voxel min for CT scans (less are clipped before scaling).
    :param int a_max: intensity voxel max for CT scans (more are clipped before scaling).
    :param int array spatial_size: output size (in voxel) to which each image and label volume will be resized. AKA input size for the neural network.
    :param int batch_size: adjyst batch size, default is 1.
    :return PyTorch DataLoader objects: used to train neural network.
"""
def preprocess(pixdim=params['pixdim'], a_min=params['a_min'], a_max=params['a_max'], spatial_size=params['spatial_size'], batch_size=params['batch_size']):
    train_files, test_files = prepare()

    # reproduce training results
    set_determinism(seed=0)

    # and apply transformations to them
    train_transforms = Compose([
        LoadImaged(keys=["vol", "seg"]),
        EnsureChannelFirstD(keys=["vol", "seg"]),
        Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
        Orientationd(keys=["vol", "seg"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["vol", "seg"], source_key="vol"),
        Resized(keys=["vol", "seg"], spatial_size=spatial_size),
        ToTensord(keys=["vol", "seg"]),
    ])

    # transforms for test data
    test_transforms = Compose([
        LoadImaged(keys=["vol", "seg"]),
        EnsureChannelFirstD(keys=["vol", "seg"]),
        Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
        Orientationd(keys=["vol", "seg"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["vol", "seg"], source_key="vol"),
        Resized(keys=["vol", "seg"], spatial_size=spatial_size),
        ToTensord(keys=["vol", "seg"]),
    ])
    
    train_ds = Dataset(data=train_files, transform=train_transforms)
    test_ds = Dataset(data=test_files, transform=test_transforms)

    train_loader = DataLoader(train_ds, batch_size=batch_size)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    return train_loader, test_loader
